In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Staryy Nadym,RU,2021-09-09 05:59:11,65.6118,72.6842,50.72,49,10,10.67
1,1,Sandur,IN,2021-09-09 05:59:11,15.1000,76.5500,79.52,66,93,19.69
2,2,Vaini,TO,2021-09-09 05:59:11,-21.2000,-175.2000,69.96,94,90,11.50
3,3,Kiunga,PG,2021-09-09 05:59:12,-6.1219,141.2906,80.17,91,98,4.81
4,4,Zhigansk,RU,2021-09-09 05:59:12,66.7697,123.3711,54.77,56,52,3.13


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [16]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.

fig = gmaps.figure()
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig


Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
#clouds = city_data_df["Cloudiness"]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [19]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Sandur,IN,2021-09-09 05:59:11,15.1000,76.5500,79.52,66,93,19.69
3,3,Kiunga,PG,2021-09-09 05:59:12,-6.1219,141.2906,80.17,91,98,4.81
8,8,Pundaguitan,PH,2021-09-09 05:59:14,6.3711,126.1689,85.28,69,97,16.80
9,9,Salalah,OM,2021-09-09 05:59:14,17.0151,54.0924,80.01,88,100,9.06
11,11,Hithadhoo,MV,2021-09-09 05:59:15,-0.6000,73.0833,83.12,69,75,4.45
12,12,Sibu,MY,2021-09-09 05:59:16,2.3000,111.8167,88.11,66,40,3.44
13,13,Georgetown,MY,2021-09-09 05:58:06,5.4112,100.3354,87.73,74,20,3.44
16,16,Tabas,IR,2021-09-09 05:59:17,33.5959,56.9244,86.04,20,0,3.33
19,19,Luba,GQ,2021-09-09 05:59:17,3.4568,8.5547,75.09,85,83,0.92
22,22,Pascagoula,US,2021-09-09 05:59:18,30.3658,-88.5561,76.41,98,99,6.85


In [20]:
preferred_cities_df.count()

City_ID       194
City          194
Country       194
Date          194
Lat           194
Lng           194
Max Temp      194
Humidity      194
Cloudiness    194
Wind Speed    194
dtype: int64

In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Sandur,IN,79.52,15.1000,76.5500,
3,Kiunga,PG,80.17,-6.1219,141.2906,
8,Pundaguitan,PH,85.28,6.3711,126.1689,
9,Salalah,OM,80.01,17.0151,54.0924,
11,Hithadhoo,MV,83.12,-0.6000,73.0833,
12,Sibu,MY,88.11,2.3000,111.8167,
13,Georgetown,MY,87.73,5.4112,100.3354,
16,Tabas,IR,86.04,33.5959,56.9244,
19,Luba,GQ,75.09,3.4568,8.5547,
22,Pascagoula,US,76.41,30.3658,-88.5561,


In [25]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
  
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))